<a href="https://colab.research.google.com/github/quaneh/portfolio/blob/main/NeMo_Guardrails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NeMo Guardrails

In this notebook I will be investigating how to use NeMo Guardrails to create AI chatbots. I'll be following along with some tutorials from the wonderful James Briggs, please check out his videos on his chanel: https://www.youtube.com/@jamesbriggs/videos

I've made some changes to do things differently to the approach in the tutorial.
In this case we'll be trying to ensure that the chatbot doesn't give fitness enthusiasts any advice about supplements they should take. As this might open us up to some liability claims for providing medical advice.

In [2]:
!pip install -qU nemoguardrails openai langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 21.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 7.8 MB

In [3]:
from getpass import getpass

OPENAI_API_KEY = getpass()

··········


In [4]:
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

As I'm running this notebook in Colab, I'll import the config from a string. However, this config can be found in config/config.yml  and config/topics.co
These are our colang file and our config file.

In [95]:
yaml_content =  """
models:
- type: main
  engine: openai
  model: gpt-3.5-turbo-instruct
"""

colang_content = """
define user give name
    "My name is Plato"
    "I'm Aristotle"
    "I am Johnny"


define user greeting
    "hello"
    "hi"
    "what's up?"

define user ask supplements
    "How much creatine should I take?"
    "What is the best dosage of ZMA?"
    "Does Anavar have side effects?"
    "Testosterone injection sites"
    "Omega-3 supplementation"

define user ask time
    "What time is it?"
    "What is the time right now?"
    "What is the current time?"

define bot name greeting
    "Hey $name!"

define bot answer supplements
    "I'm just a chatbot. I can't give you any information about supplements."

define flow give name
    user give name
    $name = ...
    bot name greeting

define flow
    user greeting
    if $name
        bot name greeting
    else
        bot ask name

define flow supplements
    user ask supplements
    bot answer supplements
    bot offer help

define flow
    user ask time
    $answer = execute what_time()
    bot $answer
"""

In [96]:

from nemoguardrails import LLMRails, RailsConfig

config = RailsConfig.from_content(
    yaml_content=yaml_content,
    colang_content=colang_content
)

rails = LLMRails(config)

We'll start by just passing simple promts to our rails, to see how they react.

We can see that when we ask a question about supplements, we're directed to the supplement flow.
However, the bot then contradicts itself, and offers to seach for that information anyway!!

In [97]:
res = await rails.generate_async(prompt='hey there')
print(res)

Hey there! Before we continue, may I know your name?


In [98]:
res = await rails.generate_async(prompt='How much creatine should I take?')
print(res)

I'm just a chatbot. I can't give you any information about supplements.
But I can help you search for reliable sources or connect you with a nutritionist who can give you personalized advice. Would you like me to do that for you?


Let's see if we can fix this behaviour by providing a more strict answer in the guardrails:

We'll change the answer from:
"I'm just a chatbot. I can't give you any information about supplements."

to:

"Sorry, I can't give you any information about supplements. You should speak to a doctor instead."

In [99]:
colang_content = """
define user give name
    "My name is Brian"
    "I'm Fred"
    "I am Stephen"

define user greeting
    "hello"
    "hi"
    "what's up?"

define user ask supplements
    "How much creatine should I take?"
    "What is the best dosage of ZMA?"
    "Does Anavar have side effects?"
    "Testosterone injection sites"
    "Omega-3 supplementation"

define user ask time
    "What time is it?"
    "What is the time right now?"
    "What is the current time?"

define bot name greeting
    "Hey $name!"

define bot answer supplements
    "Sorry, I can't give you any information about supplements. You should speak to a doctor instead."

define flow give name
    user give name
    $name = ...
    bot name greeting

define flow
    user greeting
    if $name
        bot name greeting
    else
        bot ask name

define flow supplements
    user ask supplements
    bot answer supplements
    bot offer help

define flow
    user ask time
    $answer = execute what_time()
    bot $answer
"""

In [100]:
config = RailsConfig.from_content(
    yaml_content=yaml_content,
    colang_content=colang_content
)

rails = LLMRails(config)

In [101]:
res = await rails.generate_async(prompt='How much creatine should I take?')
print(res)

Sorry, I can't give you any information about supplements. You should speak to a doctor instead.
Is there anything else you need assistance with?


Great!! Our firmer response has had the desired result.

Let's check whether our chatbot follows the other flows well.
We'll send it a greeting, and see if it asks us our name and provides the appropriate response.

In [102]:
messages = [
    {"role": "context", "content": ""},
    {"role": "user", "content": "Hey there!"}
]

In [103]:
res = await rails.generate_async(messages=messages)
res

{'role': 'assistant', 'content': 'Hello there! May I know your name please?'}

In [104]:
messages += [
    res,
    {"role": "user", "content": "I'm Marko"}
]

In [105]:
res = await rails.generate_async(messages=messages)
res

{'role': 'assistant', 'content': 'Hey !'}

It's followed the right flow, but it seems like it's been unable to correctly extract the correct user name variable.

Experimenting with different models produces wildly different results.

We can also tell Guardrails to use functions that we have predefined.

Let's write a short function that tells the user what time it is.

In [73]:
import datetime

async def what_time():

    time = datetime.datetime.now()
    return(f'It is now {time}')

In [74]:
await what_time()

'It is now 2024-04-15 09:34:35.949415'

In [75]:
rails.register_action(action=what_time, name="what_time")

In [76]:
await rails.generate_async(prompt="What time is it?")

'It is now 2024-04-15 09:34:36.987967'